%pip install keras_tuner
%pip install tensorflow
%pip install yfinance
%pip install pandas_ta
%pip install plotly
%pip install scikit-learn
%pip install pyarrow
%pip install fastparquet
%pip install imbalanced-learn
%pip install numpy --upgrade

%pip install numpy --upgrade

In [13]:
print("Importa librerie", end="", flush=True)
from keras_tuner import HyperModel, RandomSearch, Hyperband, BayesianOptimization
print(".", end="", flush=True)
from tensorflow.keras.layers import LSTM, RepeatVector, Dropout, TimeDistributed, Dense
print(".", end="", flush=True)
from tensorflow.keras.models import Sequential
print(".", end="", flush=True)
from tensorflow.keras.regularizers import l2
print(".", end="", flush=True)
from tensorflow.keras import metrics
print(".", end="", flush=True)
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
print(".", end="", flush=True)
from tensorflow.keras.optimizers import Adam
print(".", end="", flush=True)
import pandas as pd
print(".", end="", flush=True)
import yfinance as yf
print(".", end="", flush=True)
import numpy as np
print(".", end="", flush=True)
import tensorflow as tf
print(".", end="", flush=True)
import funzioni as fx
print(".", end="", flush=True)
from sklearn.preprocessing import PowerTransformer
print(".", end="", flush=True)
from sklearn.base import clone
print(".", end="", flush=True)
import pickle
print(".", end="", flush=True)
import gc
print(".", end="", flush=True)
gc.enable()
import os
print("=", end="\n", flush=True)
print("ok")

Importa librerie................=
ok


In [14]:
simbolo_test = "BTG"
simbolo_validazione = "DHT"
n_simboli_addestramento = "Tutti"
epochs=50
batch_size=5000
n_timesteps = 60 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 1 # n. barre nel futuro di cui si desidera prevedere il prezzo
set_file_x_y = f"_{n_simboli_addestramento}"
initial_lr = 0.001

features_prezzo = [
    "Close",
#    "EMA_5", 
    "EMA_20", 
    "EMA_50",
    "Open",  
    "High",
    "Low",
    "PSAR",
    "SUPERT", 
]

features_da_scalare_singolarmente = [
    "Volume",
    "ATR",
    "PSARaf",
    "ADX",
]

features_meno_piu = [
    "MACDh",    
    "AROONOSC",
    "TRIX",
    "TRIXs",
    "DM_OSC",
]

features_no_scala = [
    "SUPERTd",  
    "PSARr"
]

features_candele = [
#    "CDL_2CROWS", "CDL_3BLACKCROWS", "CDL_3INSIDE", "CDL_3LINESTRIKE", "CDL_3OUTSIDE", "CDL_3STARSINSOUTH", "CDL_3WHITESOLDIERS", "CDL_ABANDONEDBABY", "CDL_ADVANCEBLOCK", "CDL_BELTHOLD", "CDL_BREAKAWAY", "CDL_CLOSINGMARUBOZU", "CDL_CONCEALBABYSWALL", "CDL_COUNTERATTACK", "CDL_DARKCLOUDCOVER", "CDL_DOJI_10_0.1", "CDL_DOJISTAR", "CDL_DRAGONFLYDOJI", "CDL_ENGULFING", "CDL_EVENINGDOJISTAR", "CDL_EVENINGSTAR", "CDL_GAPSIDESIDEWHITE", "CDL_GRAVESTONEDOJI", "CDL_HAMMER", "CDL_HANGINGMAN", "CDL_HARAMI", "CDL_HARAMICROSS", "CDL_HIGHWAVE", "CDL_HIKKAKE", "CDL_HIKKAKEMOD", "CDL_HOMINGPIGEON", "CDL_IDENTICAL3CROWS", "CDL_INNECK", "CDL_INSIDE", "CDL_INVERTEDHAMMER", "CDL_KICKING", "CDL_KICKINGBYLENGTH", "CDL_LADDERBOTTOM", "CDL_LONGLEGGEDDOJI", "CDL_LONGLINE", "CDL_MARUBOZU", "CDL_MATCHINGLOW", "CDL_MATHOLD", "CDL_MORNINGDOJISTAR", "CDL_MORNINGSTAR", "CDL_ONNECK", "CDL_PIERCING", "CDL_RICKSHAWMAN", "CDL_RISEFALL3METHODS", "CDL_SEPARATINGLINES", "CDL_SHOOTINGSTAR", "CDL_SHORTLINE", "CDL_SPINNINGTOP", "CDL_STALLEDPATTERN", "CDL_STICKSANDWICH", "CDL_TAKURI", "CDL_TASUKIGAP", "CDL_THRUSTING", "CDL_TRISTAR", "CDL_UNIQUE3RIVER", "CDL_UPSIDEGAP2CROWS", "CDL_XSIDEGAP3METHODS",
]

elenco_targets = [
#    "EMA_5",
#    "EMA_20", 
#    "EMA_50",
    #"Open",
    #"High",
    #"Low",
    "Target"
]

col_features_prezzo = {col: idx for idx, col in enumerate(features_prezzo)}
col_features_da_scalare_singolarmente = {col: idx for idx, col in enumerate(features_da_scalare_singolarmente)}
col_features_meno_piu = {col: idx for idx, col in enumerate(features_meno_piu)}
col_features_no_scala = {col: idx for idx, col in enumerate(features_no_scala)}
col_features_candele = {col: idx for idx, col in enumerate(features_candele)}
col_targets = {col: idx for idx, col in enumerate(elenco_targets)}
n_features = len(col_features_prezzo) + len(col_features_da_scalare_singolarmente) + len(col_features_meno_piu) + len(col_features_no_scala) + len(col_features_candele) 
n_targets = len(col_targets) 

class MixedHyperModel(HyperModel):
    def __init__(self, n_timesteps, n_features, n_targets, giorni_previsione):
        self.n_timesteps = n_timesteps
        self.n_features = n_features
        self.n_targets = n_targets
        self.giorni_previsione = giorni_previsione

    def build(self, hp):
        model = Sequential()

        # Layer LSTM iniziale
        model.add(LSTM(hp.Int('lstm_units_1', 50, 500, step=50),
                       input_shape=(self.n_timesteps, self.n_features), kernel_initializer='glorot_uniform'))
        model.add(RepeatVector(self.giorni_previsione))

        # Aggiunta di layer LSTM intermedi
        for i in range(hp.Int('num_lstm_layers', 1, 4)):
            model.add(LSTM(hp.Int(f'lstm_units_{i+2}', 50, 500, step=50), return_sequences=True,
                           kernel_regularizer=l2(hp.Float(f'l2_rate_{i+2}', 0.000001, 5, step=0.001)), kernel_initializer='glorot_uniform'))
            model.add(Dropout(hp.Float(f'lstm_dropout_{i+2}', 0, 0.5, step=0.1)))

        # Aggiunta di layer Dense
        for i in range(hp.Int('num_dense_layers', 1, 4)):
            model.add(TimeDistributed(Dense(hp.Int(f'dense_units_{i}', 50, 500, step=50), activation='relu', kernel_initializer='glorot_uniform')))
            model.add(Dropout(hp.Float(f'dense_dropout_{i}', 0, 0.5, step=0.1)))

        model.add(TimeDistributed(Dense(1, activation='sigmoid', kernel_initializer='glorot_uniform')))

        model.compile(optimizer="adam", 
              loss='binary_crossentropy', 
              metrics=['accuracy', 
                       metrics.Precision(name='precision'), 
                       metrics.Recall(name='recall'), 
                       metrics.AUC(name='auc')])
        
        return model

input_shape = (n_timesteps, n_features)
output_shape = (giorni_previsione, n_targets)
hypermodel = MixedHyperModel(n_timesteps, n_features, n_targets, giorni_previsione)
print("ok")


ok


In [15]:
if not os.path.exists('dati'):
    os.makedirs('dati')

if os.path.exists("lista_ticker.parquet"):
    print("Caricamento lista_ticker esistente", flush=True)
    lista_ticker = pd.read_parquet("lista_ticker.parquet")
else:
    print("Download lista ticker", flush=True)
    lista_ticker = pd.read_parquet("Tickers_De_Giro.parquet")
    lista_ticker = lista_ticker.sample(frac=1).reset_index(drop=True)
    lista_ticker = lista_ticker.loc[(lista_ticker["Ticker"] != simbolo_test) & (lista_ticker["Ticker"] != simbolo_validazione) & (lista_ticker["Categoria"] != "D"), :]
    lista_ticker.to_parquet("lista_ticker.parquet")

if n_simboli_addestramento == 'Tutti': 
    n_simboli_addestramento = len(lista_ticker)

if os.path.exists("dati/ticker_val.parquet"):
    print("Caricamento dati ticker validazione", flush=True)
    ticker_val = pd.read_parquet("dati/ticker_val.parquet")
else:
    print("Download dati ticker validazione")
    ticker_val = yf.download(simbolo_validazione, start='2010-01-01', end='2023-12-31', progress=False)
    ticker_val.index = ticker_val.index.date
    print("Calcolo indicatori ticker validazione")
    ticker_val = fx.crea_indicatori(ticker_val)
    ticker_val.dropna(axis=0, inplace=True)
    ticker_val.to_parquet("dati/ticker_val.parquet")

print("Definizione features e target ticker validazione", flush=True)
idx_val, X_val, Y_val, _ = fx.to_XY(ticker_val, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)

if os.path.exists("dati/indice.npy"):    
    inizio = np.load("dati/indice.npy")
    inizio += 1
else:
    inizio = 0
print("ok")

Caricamento lista_ticker esistente
Caricamento dati ticker validazione
Definizione features e target ticker validazione
ok


In [16]:
def set_di_tuning():
    if os.path.exists(f"dati/X{set_file_x_y}.npy") and os.path.exists(f"dati/Y{set_file_x_y}.npy"):
        print("Caricamento X e Y")
        X = np.load(f'dati/X{set_file_x_y}.npy')
        Y = np.load(f'dati/Y{set_file_x_y}.npy')
    else:
        X = np.zeros((0, n_timesteps, n_features))
        #Y = np.zeros((0, giorni_previsione, n_targets)) #togliere in caso di classificazione binaria
        Y = np.zeros((0, 1)) #solo per classificazione binaria
    if inizio < n_simboli_addestramento:
        lista_x, lista_y = [], []
        for i_ticker in range (inizio, n_simboli_addestramento):
            nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
            print(f"\033[48;5;42m{i_ticker+1} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
            print("Download dati ticker", flush=True)
            try:
                ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
                if ticker["Close"].iloc[-1] >= 1:
                    ticker.index = ticker.index.date
                    print("Calcolo indicatori ticker", flush=True)
                    ticker = fx.crea_indicatori(ticker)
                    ticker.dropna(axis=0, inplace=True)

                    print("Definizione features e target", flush=True)
                    idx, X_train, Y_train, _ = fx.to_XY(ticker, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)

                    print("Aggiunta dati a liste X e Y", flush=True)
                    lista_x.append(X_train)        
                    lista_y.append(Y_train)   
                    if (i_ticker % 100 == 0) and (i_ticker > 0):
                        print("Concatenamento su X, Y", flush=True)
                        X_arr = np.vstack(lista_x)
                        Y_arr = np.vstack(lista_y)
                        X = np.vstack((X, X_arr))
                        Y = np.vstack((Y, Y_arr))
                        print("Salvataggio X e Y su file", flush=True)
                        np.save(f'dati/X{set_file_x_y}', X)
                        np.save(f'dati/Y{set_file_x_y}', Y)
                        np.save("dati/indice", i_ticker)
                        lista_x, lista_y = [], []
                    gc.collect()
            except Exception as e:
                print(e)
                gc.collect()
                continue
                
        print("Concatenamento su X, Y", flush=True)
        X_arr = np.vstack(lista_x)
        Y_arr = np.vstack(lista_y)
        X = np.vstack((X, X_arr))
        Y = np.vstack((Y, Y_arr))
        print("Salvataggio X e Y su file", flush=True)
        np.save(f'dati/X{set_file_x_y}', X)
        np.save(f'dati/Y{set_file_x_y}', Y)
        np.save("dati/indice", i_ticker)
        lista_x, lista_y = [], []

        gc.collect()
#set_di_tuning()
print("ok")

ok


In [17]:
def rd(hypermodel, X_train, Y_train, X_val, Y_val):
    random_tuner = RandomSearch(
        hypermodel,
        objective='val_loss',
        max_trials=50,
        executions_per_trial=2,
        directory='random_search',
        project_name='random_tuning'
    )
    random_tuner.search_space_summary()
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    random_tuner.search(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    # Ottieni il miglior modello e i migliori iperparametri
    best_model_random = random_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_random = random_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Random Search:", best_hyperparameters_random.values)
    
    return best_model_random, best_hyperparameters_random
print("ok")

ok


In [18]:
def hb(hypermodel, X_train, Y_train, X_val, Y_val):
    hyperband_tuner = Hyperband(
        hypermodel,
        objective='val_loss',
        max_trials=50,
        directory='hyperband_search',
        project_name='hyperband_tuning'
    )
    
    hyperband_tuner.search_space_summary()
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    hyperband_tuner.search(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    
    best_model_hyperband = hyperband_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_hyperband = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Hyperband:", best_hyperparameters_hyperband.values)
    
    return best_model_hyperband, best_hyperparameters_hyperband
print("ok")

ok


In [19]:
def bay(hypermodel, X_train, Y_train, X_val, Y_val):
    bayesian_tuner = BayesianOptimization(
        hypermodel,
        objective='val_loss',
        max_trials=50,
        directory='bayesian_search',
        project_name='bayesian_tuning'
    )
    
    bayesian_tuner.search_space_summary()
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    bayesian_tuner.search(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    
    best_model_bayesian = bayesian_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_bayesian = bayesian_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Bayesian Optimization:", best_hyperparameters_bayesian.values)
    
    return best_model_bayesian, best_hyperparameters_bayesian
print("ok")

ok


In [20]:
#print("Tuning")
#model, par = bay(hypermodel, X, Y, X_val, Y_val)
#print("ok")

In [21]:
#model.save("tuning.keras")
#with open('hyperparameters.pkl', 'wb') as f:
    #pickle.dump(par, f)

In [22]:
def addestramento():
    from tensorflow.keras.models import load_model

    model = load_model("tuning.keras")
    
    if os.path.exists("dati/indice_addestramento.npy"):    
        inizio = np.load("dati/indice_addestramento.npy")
    else:
        inizio = 0
    print("ok")
    X = np.zeros((0, n_timesteps, n_features))
    #Y = np.zeros((0, giorni_previsione, n_targets)) #togliere in caso di classificazione binaria
    Y = np.zeros((0, 1)) #solo per classificazione binaria
    if inizio < n_simboli_addestramento:
        for i_ticker in range (inizio, n_simboli_addestramento):
            nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
            print(f"\033[48;5;42m{i_ticker+1} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
            print("Download dati ticker", flush=True)
            try:
                ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
                if ticker["Close"].iloc[-1] >= 1:
                    ticker.index = ticker.index.date
                    print("Calcolo indicatori ticker", flush=True)
                    ticker = fx.crea_indicatori(ticker)
                    ticker.dropna(axis=0, inplace=True)

                    print("Definizione features e target", flush=True)
                    _, X_train, Y_train, _ = fx.to_XY(ticker, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)

                    print(f"Addestramento simbolo {nome_simbolo}", flush=True)
                    model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val))
                    np.save("dati/indice_addestramento", i_ticker)

                    gc.collect()
            except Exception as e:
                print(e)
                gc.collect()
                continue

addestramento()